<a href="https://colab.research.google.com/github/johntango/1125exercise01/blob/master/pusubDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
GCP_PROJECT_ID = "demoproject001-330218"
PROJECT_NUMBER = '429307294394'

In [6]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

AuthorizationError: ignored

In [3]:
 !pip install google-cloud-pubsub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 233 kB 4.2 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 4.4 MB 54.3 MB/s 
     |████████████████████████████████| 1.0 MB 58.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.1
    Uninstalling grpcio-1.46.1:
      Successfully uninstalled grpcio-1.46.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [3]:
# environment variable setup for private key file
import time
from concurrent.futures import TimeoutError
from google.cloud import pubsub_v1
import json
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/demoproject001-330218-4160b5cbaabb.json"


# GCP topic, project & subscription ids
PUB_SUB_TOPIC = "MyTopic001"
PUB_SUB_PROJECT = "demoproject001-330218"
PUB_SUB_SUBSCRIPTION = "MyTopic001-sub"

# Pub/Sub consumer timeout
timeout = 3.0

# callback function for processing consumed payloads
# prints recieved payload


def process_payload(message):
    print(f"Received {message.data}.")
    message.ack()

# producer function to push a message to a topic


def push_payload(payload, topic, project):
    publisher = pubsub_v1.PublisherClient()
    topic_path = publisher.topic_path(project, topic)
    data = json.dumps(payload).encode("utf-8")
    future = publisher.publish(topic_path, data=data)
    print("Pushed message to topic.")

# consumer function to consume messages from a topics for a given timeout period


def consume_payload(project, subscription, callback, period):
    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(project, subscription)
    print(f"Listening for messages on {subscription_path}..\n")
    streaming_pull_future = subscriber.subscribe(
        subscription_path, callback=callback)
    # Wrap subscriber in a 'with' block to automatically call close() when done.
    with subscriber:
        try:
            # When `timeout` is not set, result() will block indefinitely,
            # unless an exception is encountered first.
            streaming_pull_future.result(timeout=period)
        except TimeoutError:
            streaming_pull_future.cancel()


# loop to test producer and consumer functions with a 3 second delay
while(True):
    print("===================================")
    payload = {"data": "Payload data", "timestamp": time.time()}
    print(f"Sending payload: {payload}.")
    push_payload(payload, PUB_SUB_TOPIC, PUB_SUB_PROJECT)
    consume_payload(PUB_SUB_PROJECT, PUB_SUB_SUBSCRIPTION,
                    process_payload, timeout)
    time.sleep(3)


Sending payload: {'data': 'Payload data', 'timestamp': 1653415720.8312664}.
Pushed message to topic.
Listening for messages on projects/demoproject001-330218/subscriptions/MyTopic001-sub..



Failed to publish 1 messages.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/google/api_core/grpc_helpers.py", line 67, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/grpc/_channel.py", line 946, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/usr/local/lib/python3.7/dist-packages/grpc/_channel.py", line 849, in _end_unary_response_blocking
    raise _InactiveRpcError(state)
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.PERMISSION_DENIED
	details = "User not authorized to perform this action."
	debug_error_string = "{"created":"@1653415721.180395128","description":"Error received from peer ipv4:142.251.8.95:443","file":"src/core/lib/surface/call.cc","file_line":952,"grpc_message":"User not authorized to perform this action.","grpc_status":7}"
>

The above exception was the direct cause of the f

PermissionDenied: ignored